# 💌NLP Sentiment Analysis Dating Apps Beoordelingen | 2017-2023 💌

# Inleiding

Het hoofddoel van dit project is het analyseren van beoordelingen van dating-apps uit de Google Play Store in verschillende landen (India en de VS), beide grote markten met een breed netwerk van gebruikers van dating-apps. Door deze datasets samen te voegen, zal ik een NLP-sentimentanalyse uitvoeren om de verschillende gedragingen en algemene gevoelens van gebruikers op verschillende datingplatforms te onderzoeken.

De analyse richt zich op het beantwoorden van drie hoofdvragen:

- Wat zijn de gebruikelijke gedragingen en gevoelens die mensen hebben bij het gebruik van verschillende dating-apps (Tinder, Bumble, Hinge)?
- Welke dating-app is het meest geschikt voor daten, relaties of beste functionaliteiten?
- Wat is het algemene gevoel van gebruikers (in algemeen) over het gebruik van dating-apps?

Aan het einde van deze notebook zal een samenvatting staan van de inzichten uit de sentimentanalyse. De verschillende datasets die ik voor deze analyse heb gebruikt, zijn:


- Dating Apps Reviews India 2017-2022 (dating_review): https://www.kaggle.com/datasets/sidharthkriplani/datingappreviews/data
- Dating Apps Reviews US 2023 (dating_review2): https://figshare.com/articles/dataset/Text_of_user_reviews_of_dating_apps/21895827

# 📦Packages laden

In dit deel van de code worden alle benodigde bibliotheken en pakketten geïmporteerd om een NLP-sentimentanalyse op dating-app beoordelingen uit te voeren. Hier is een overzicht van de belangrijkste geïmporteerde pakketten:
- NLTK: Natural Language Toolkit, gebruikt voor tekstverwerkingstaken zoals tokenisatie, stemming, en lemmatizatie.
- FuzzyWuzzy en RapidFuzz: Voor fuzzy matching, wat handig is voor het vergelijken van teksten die niet exact overeenkomen.
- Multiprocessing: Om processen te versnellen door parallelle verwerking mogelijk te maken

In [2]:
#import libraries
import pandas as pd
import numpy as np
import string
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import seaborn as sns
%matplotlib inline
import re
from sklearn.naive_bayes import BernoulliNB
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import joblib
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from multiprocessing import Pool, cpu_count
from fuzzywuzzy import process
from rapidfuzz import process
# Download necessary NLTK data files
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kaile\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaile\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kaile\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 🔋Datasets Laden

In dit gedeelte van de code worden twee datasets met beoordelingen van dating-apps ingeladen. De datasets worden ingelezen met behulp van de pd.read_csv() functie van de pandas-bibliotheek, waarbij het pad naar het CSV-bestand wordt opgegeven. Het eerste bestand bevat gebruikersbeoordelingen van verschillende dating-apps uit de periode 2017-2023 uit India en het tweede bestand bevat gebruikersbeoordelingen van verschillende dating-apps uit de periode 2023 uit VS.

In [3]:
#Dataset 'dating app reviews India 2017-2023' laden 
dating_review= pd.read_csv("C:\\Users\\kaile\\OneDrive\\Documents\\S4\\notebooks\\DatingAppReviewsDataset.csv\\DatingAppReviewsDataset.csv",index_col=0)
dating_review

,Name,Review,Rating,#ThumbsUp,Date&Time,App
0,linah sibanda,On this app i cant find a partner,5,0,18-02-2022 01:19,Tinder
1,Norman Johnson,Tinder would be so much better if we could spe...,3,0,18-02-2022 01:16,Tinder
2,David Hume,Still doesn't correctly notify matches or mess...,1,0,18-02-2022 01:11,Tinder
3,Last 1 Standing,"Got banned because I updated my bio to say ""I ...",2,0,18-02-2022 01:11,Tinder
4,Arthur Magamedov,Love it!,5,0,18-02-2022 01:06,Tinder
...,...,...,...,...,...,...
52989,A Google user,Useless - I'm in the UK and it tells me i'm ov...,2,5,12-07-2017 01:44,Hinge
52990,Brian Shook,I can't get past the initial set up. It won't...,1,11,12-07-2017 01:36,Hinge
52991,A Google user,This is incredible! A quality dating app for A...,5,1,12-07-2017 01:32,Hinge
52992,A Google user,"""Over Water"" ... Can't choose location.",2,8,12-07-2017 01:28,Hinge


In [4]:
# Dataset 'dating app reviews US 2023' laden
dating_review2 = pd.read_csv("C:\\Users\\kaile\\all_reviews_combined.csv")
display(dating_review2)

,Name,Review,Rating,#ThumbsUp,Date&Time,App
0,Tina Richardson,Matches can be misleading. I keep getting matc...,2.0,0.0,25-03-2023 22:24,Tinder
1,Rebekah Morales,This app is not working. Whenever i got notifi...,1.0,0.0,21-09-2023 02:07,Tinder
2,Adam Pierce,They want you to pay,1.0,0.0,04-05-2023 22:21,Tinder
3,Yvette Tran,BANNED ME FOR NO REASON AND I DIDN'T EVEN GET ...,1.0,0.0,21-04-2023 14:24,Tinder
4,Kyle Clark,Used to be good and easy to use... Every time ...,2.0,0.0,11-09-2023 20:40,Tinder
...,...,...,...,...,...,...
703174,Michael Garcia,Useless - I'm in the UK and it tells me i'm ov...,2.0,5.0,26-06-2023 11:27,Hinge
703175,Matthew Barnes,I can't get past the initial set up. It won't...,1.0,11.0,03-01-2023 10:10,Hinge
703176,Mr. Paul Long,This is incredible! A quality dating app for A...,5.0,1.0,12-11-2023 21:20,Hinge
703177,David Gomez,"""Over Water"" ... Can't choose location.",2.0,8.0,29-10-2023 17:37,Hinge


# 🧹Merging & Data- Cleaning Stappen

In de eerste codeblok worden de kolommen Rating en #ThumbsUp in beide datasets (dating_review en dating_review2) omgezet naar numerieke waarden van het type Int32. De pd.to_numeric functie wordt gebruikt met errors='coerce', wat ervoor zorgt dat niet-numerieke waarden worden omgezet naar NaN, gevolgd door conversie naar een integer type.

In [5]:
# Convert 'Rating' and '#ThumbsUp' columns to int64
dating_review['Rating'] = pd.to_numeric(dating_review['Rating'], errors='coerce').astype('Int32')
dating_review['#ThumbsUp'] = pd.to_numeric(dating_review['#ThumbsUp'], errors='coerce').astype('Int32')

# Convert 'Rating' and '#ThumbsUp' columns to int64
dating_review2['Rating'] = pd.to_numeric(dating_review2['Rating'], errors='coerce').astype('Int32')
dating_review2['#ThumbsUp'] = pd.to_numeric(dating_review2['#ThumbsUp'], errors='coerce').astype('Int32')

In de volgende stap worden duplicaten verwijderd uit zowel de originele dataset (dating_review) als de tweede dataset (dating_review2). Dit voorkomt dat dezelfde beoordelingen meerdere keren in de gecombineerde dataset voorkomen.

In [6]:
# Remove duplicates from the original dating review dataset
dating_review = dating_review.drop_duplicates()

# Remove duplicates from the unified dating app reviews dataset
dating_review2= dating_review2.drop_duplicates()

 Daarna als laatste stap worden de twee datasets (dating_review en dating_review2) verticaal samengevoegd met de pd.concat functie. Door ignore_index=True te gebruiken, wordt de index opnieuw gegenummerd in de samengevoegde dataset. Vervolgens wordt de samengevoegde dataset weergegeven met de display() functie.

In [7]:
# Concatenate the datasets vertically
merged_reviews = pd.concat([dating_review, dating_review2], ignore_index=True)

# Display the merged dataset
display(merged_reviews)


,Name,Review,Rating,#ThumbsUp,Date&Time,App
0,linah sibanda,On this app i cant find a partner,5,0,18-02-2022 01:19,Tinder
1,Norman Johnson,Tinder would be so much better if we could spe...,3,0,18-02-2022 01:16,Tinder
2,David Hume,Still doesn't correctly notify matches or mess...,1,0,18-02-2022 01:11,Tinder
3,Last 1 Standing,"Got banned because I updated my bio to say ""I ...",2,0,18-02-2022 01:11,Tinder
4,Arthur Magamedov,Love it!,5,0,18-02-2022 01:06,Tinder
...,...,...,...,...,...,...
1385168,Michael Garcia,Useless - I'm in the UK and it tells me i'm ov...,2,5,26-06-2023 11:27,Hinge
1385169,Matthew Barnes,I can't get past the initial set up. It won't...,1,11,03-01-2023 10:10,Hinge
1385170,Mr. Paul Long,This is incredible! A quality dating app for A...,5,1,12-11-2023 21:20,Hinge
1385171,David Gomez,"""Over Water"" ... Can't choose location.",2,8,29-10-2023 17:37,Hinge


# 📃Data omschrijven

De data is afkomstig uit verschillende bronnen, voornamelijk de Google Play Store. Elke beoordeling is gekoppeld aan een specifieke dating-app waarmee de gebruiker interactie had, samen met een bijbehorende beoordeling en, indien beschikbaar, een ThumbsUp-score om aan te geven of anderen het commentaar nuttig vonden of ermee instemden. De dataset is gestructureerd met de volgende kolommen:

- Name: De naam van de gebruiker die de review heeft geschreven.
- Review: De tekst van de review die de gebruiker heeft achtergelaten.
- Rating: De beoordeling gegeven door de gebruiker (waarschijnlijk op een schaal van 1-5).
- #ThumbsUp: Aantal "duimpjes omhoog" of likes die de review heeft gekregen.
- Date&Time: De datum en tijd waarop de review is geplaatst.
- App: De specifieke dating-app die de gebruiker beoordeeld heeft (bijvoorbeeld Tinder of Hinge).

Opmerking: Sommige gegevens zijn gegenereerd met behulp van de "faker"-functie in Python om synthetische data te maken voor de tweede dataset, met name voor de kolommen Name en Date&Time. Deze waarden zijn dus niet echt.

# 🧼Data opschonen & Exploratory Data Analysis (EDA)

Deze stap bevat het proces van data opschoning en verkennende data-analyse (EDA) van de merged_reviews dataset. Het doel is om een goed begrip van de dataset te krijgen en ervoor te zorgen dat de gegevens in het juiste formaat en zonder missende waarden zijn voordat verdere analyse wordt uitgevoerd.

In [8]:
#Dataset laden 10 eerste rijen 
merged_reviews.head()

,Name,Review,Rating,#ThumbsUp,Date&Time,App
0,linah sibanda,On this app i cant find a partner,5,0,18-02-2022 01:19,Tinder
1,Norman Johnson,Tinder would be so much better if we could spe...,3,0,18-02-2022 01:16,Tinder
2,David Hume,Still doesn't correctly notify matches or mess...,1,0,18-02-2022 01:11,Tinder
3,Last 1 Standing,"Got banned because I updated my bio to say ""I ...",2,0,18-02-2022 01:11,Tinder
4,Arthur Magamedov,Love it!,5,0,18-02-2022 01:06,Tinder


In [9]:
# Dataset laden 10 laatste rijen 
merged_reviews.tail()

,Name,Review,Rating,#ThumbsUp,Date&Time,App
1385168,Michael Garcia,Useless - I'm in the UK and it tells me i'm ov...,2,5,26-06-2023 11:27,Hinge
1385169,Matthew Barnes,I can't get past the initial set up. It won't...,1,11,03-01-2023 10:10,Hinge
1385170,Mr. Paul Long,This is incredible! A quality dating app for A...,5,1,12-11-2023 21:20,Hinge
1385171,David Gomez,"""Over Water"" ... Can't choose location.",2,8,29-10-2023 17:37,Hinge
1385172,Richard Martinez,"My entire town counts as ""over water"" and I ca...",2,15,30-05-2023 08:53,Hinge


In [10]:
# Basisinformatie van de dataset weergeven, inclusief kolomtypes en aantal entries
merged_reviews.info()
# De vorm van de dataset (rijen, kolommen) weergeven om de dimensies beter te begrijpen
print("shape of the dataset -->>",np.shape(merged_reviews))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1385173 entries, 0 to 1385172
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   Name       1385166 non-null  object
 1   Review     1382364 non-null  object
 2   Rating     1384679 non-null  Int32 
 3   #ThumbsUp  1382910 non-null  Int32 
 4   Date&Time  1385173 non-null  object
 5   App        1385173 non-null  object
dtypes: Int32(2), object(4)
memory usage: 55.5+ MB
shape of the dataset -->> (1385173, 6)


In [11]:
# Convert 'Date&Time' column to datetime64 format
dating_review2['Date&Time'] = pd.to_datetime(dating_review2['Date&Time'], format='%d-%m-%Y %H:%M', errors='coerce')

In [12]:
#Convert 'text' column to string data type
dating_review2['Review'] = dating_review2['Review'].astype(str)

In [13]:
dating_review2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703179 entries, 0 to 703178
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Name       703179 non-null  object        
 1   Review     703179 non-null  object        
 2   Rating     702685 non-null  Int32         
 3   #ThumbsUp  700916 non-null  Int32         
 4   Date&Time  703179 non-null  datetime64[ns]
 5   App        703179 non-null  object        
dtypes: Int32(2), datetime64[ns](1), object(3)
memory usage: 28.2+ MB


In [14]:
# Convert 'Data&Time' in twee kolommen (Date)
merged_reviews["Date"] = pd.to_datetime(merged_reviews['Date&Time']).dt.date

C:\Users\kaile\AppData\Local\Temp\ipykernel_32100\774892554.py:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  merged_reviews["Date"] = pd.to_datetime(merged_reviews['Date&Time']).dt.date


In [15]:
#Convert 'Data&Time' in twee kolommen (Time)
merged_reviews["Time"] = pd.to_datetime(merged_reviews['Date&Time']).dt.time

C:\Users\kaile\AppData\Local\Temp\ipykernel_32100\1493835819.py:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  merged_reviews["Time"] = pd.to_datetime(merged_reviews['Date&Time']).dt.time


In [16]:
# Drop kolom 'Date&Time' en creër twee kolommen voor Datum & Tijd
merged_reviews= merged_reviews.drop("Date&Time",axis=1)

In [17]:
# Opnieuw informatie van de dataset weergeven om de dataconversie van 'Date&Time' te bevestigen
merged_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1385173 entries, 0 to 1385172
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   Name       1385166 non-null  object
 1   Review     1382364 non-null  object
 2   Rating     1384679 non-null  Int32 
 3   #ThumbsUp  1382910 non-null  Int32 
 4   App        1385173 non-null  object
 5   Date       1385173 non-null  object
 6   Time       1385173 non-null  object
dtypes: Int32(2), object(5)
memory usage: 66.1+ MB


In [18]:
merged_reviews.head()

,Name,Review,Rating,#ThumbsUp,App,Date,Time
0,linah sibanda,On this app i cant find a partner,5,0,Tinder,2022-02-18,01:19:00
1,Norman Johnson,Tinder would be so much better if we could spe...,3,0,Tinder,2022-02-18,01:16:00
2,David Hume,Still doesn't correctly notify matches or mess...,1,0,Tinder,2022-02-18,01:11:00
3,Last 1 Standing,"Got banned because I updated my bio to say ""I ...",2,0,Tinder,2022-02-18,01:11:00
4,Arthur Magamedov,Love it!,5,0,Tinder,2022-02-18,01:06:00


In [19]:
# Controleer op missende waarden in de dataset
# Voor elke kolom wordt het aantal ontbrekende waarden berekend en weergegeven
missing_values = merged_reviews.isnull().sum()
missing_values

Name            7
Review       2809
Rating        494
#ThumbsUp    2263
App             0
Date            0
Time            0
dtype: int64

In [20]:
# Verwijder rijen met ontbrekende waarden in de kolommen 'Name' en 'Review', we ontbreken alleen deze waarden omdat er geen
# nuttige informatie geven. Wel de kolom #ThumbsUp omdat we kunnen zien als mensen hadden dezelfde ervaring of niet.
# Door de parameter 'subset' te specificeren, worden alleen de missende waarden in deze kolommen verwijderd
merged_reviews2 = merged_reviews.dropna(subset=['Name', 'Review'])

In [21]:
# Controleer opnieuw of er nog missende waarden zijn in deze kolommen
merged_reviews_cleaned= merged_reviews2.isnull().sum()
merged_reviews_cleaned 

Name            0
Review          0
Rating        455
#ThumbsUp    2224
App             0
Date            0
Time            0
dtype: int64

In [22]:
# Statistische samenvatting van de dataset verkrijgen
# Beschrijft de statistieken zoals gemiddelde, minimum, maximum, en percentielen voor numerieke kolommen
merged_reviews.describe().T

,count,mean,std,min,25%,50%,75%,max
Rating,1384679.0,2.983099,1.763827,0.0,1.0,3.0,5.0,236.0
#ThumbsUp,1382910.0,1.908992,25.136188,0.0,0.0,0.0,0.0,5507.0


# Step-by guide

-Text Analysis -> filter words, text editing, remove special charcaters
-Confusion Matrix 
-Cloud Words 
-Predictions -> Column w emotions
-Classify Apps and predictions
-Create a column for determine the status of the user

# 💬Tekstanalyse

In [23]:
#Step 1: Display the column where text is displayed in our case the 'Review'column
# Display only the 'Review' column
print("\nSample Reviews:")
print(merged_reviews['Review']) 


Sample Reviews:
0                          On this app i cant find a partner
1          Tinder would be so much better if we could spe...
2          Still doesn't correctly notify matches or mess...
3          Got banned because I updated my bio to say "I ...
4                                                   Love it!
                                 ...                        
1385168    Useless - I'm in the UK and it tells me i'm ov...
1385169    I can't get past the initial set up.  It won't...
1385170    This is incredible! A quality dating app for A...
1385171              "Over Water" ... Can't choose location.
1385172    My entire town counts as "over water" and I ca...
Name: Review, Length: 1385173, dtype: object


In [24]:
merged_reviews['Review'] = merged_reviews['Review'].astype(str)

In [25]:
# Step 1: Tokenize the text
def tokenize_text(text):
    if not isinstance(text, str):  # Handle non-string values
        text = str(text)
    return word_tokenize(text.lower())  # Convert to lowercase and tokenize

In [26]:
# Step 2: Remove stopwords
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token not in stop_words]

In [27]:
# Step 3: Lemmatize tokens
def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

In [28]:
# Step 4: Combine tokens back into a string
def combine_tokens(tokens):
    return ' '.join(tokens)

In [29]:
# 1. Tokenize
merged_reviews['Tokens'] = merged_reviews['Review'].apply(tokenize_text)

In [30]:
# 2. Remove stopwords
merged_reviews['Filtered_Tokens'] = merged_reviews['Tokens'].apply(remove_stopwords)

In [31]:
# 3. Lemmatize
merged_reviews['Lemmatized_Tokens'] = merged_reviews['Filtered_Tokens'].apply(lemmatize_tokens)

In [32]:
# 4. Combine tokens back to a string
merged_reviews['Processed_Review'] = merged_reviews['Lemmatized_Tokens'].apply(combine_tokens)

In [33]:
# Drop intermediate columns if not needed
merged_reviews = merged_reviews.drop(['Tokens', 'Filtered_Tokens', 'Lemmatized_Tokens'], axis=1)

In [34]:
# Display the resulting DataFrame
print(merged_reviews[['Review', 'Processed_Review']].head())

                                              Review  \
0                  On this app i cant find a partner   
1  Tinder would be so much better if we could spe...   
2  Still doesn't correctly notify matches or mess...   
3  Got banned because I updated my bio to say "I ...   
4                                           Love it!   

                                    Processed_Review  
0                              app cant find partner  
1  tinder would much better could specify race lo...  
2  still n't correctly notify match message . 's ...  
3  got banned updated bio say `` feel like girl l...  
4                                             love !  
